# ColPali + Qdrant Retrieval Pipeline for a Single PDF

This notebook converts every page of a single PDF into images, embeds them with ColPali, and stores the embeddings in a local Qdrant instance for retrieval.


## 1. Setup & Imports
Install required dependencies, import modules, and configure the compute device.


In [ ]:
%pip install --upgrade --quiet torch pdf2image Pillow matplotlib tqdm qdrant-client colpali-engine transformers accelerate

import os
from pathlib import Path

import torch
from pdf2image import convert_from_path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

from colpali_engine.models import ColPali, ColPaliProcessor
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")


## 2. Configuration
Set paths and runtime parameters for the pipeline.


In [ ]:
PDF_PATH = "./data/sample.pdf"  # Update with the actual PDF path
OUTPUT_IMAGE_DIR = "./data/pdf_pages"
QDRANT_URL = "http://localhost:6333"
QDRANT_COLLECTION = "pdf_pages"
BATCH_SIZE = 2

output_dir_path = Path(OUTPUT_IMAGE_DIR)
output_dir_path.mkdir(parents=True, exist_ok=True)

print(f"PDF path: {os.path.abspath(PDF_PATH)}")
print(f"Image output directory: {output_dir_path.resolve()}")


## 3. Convert PDF Pages to Images
Render each page of the PDF to a PNG image and capture metadata for later steps.


In [ ]:
def convert_pdf_to_images(pdf_path: str, output_dir: str) -> list[dict]:
    """Convert each page of the PDF into a PNG image on disk."""
    pdf_path = Path(pdf_path)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    images = convert_from_path(str(pdf_path))

    page_records: list[dict] = []
    base_name = pdf_path.stem

    for idx, image in enumerate(images):
        file_name = f"page_{idx + 1:04d}.png"
        image_path = output_path / file_name
        image.save(image_path, format="PNG")

        page_records.append(
            {
                "page_index": idx,
                "image_path": str(image_path.resolve()),
                "pdf_file_name": base_name,
            }
        )

    return page_records


In [ ]:
pages = convert_pdf_to_images(PDF_PATH, OUTPUT_IMAGE_DIR)
print(f"Converted {len(pages)} pages to images.")

if pages:
    with Image.open(pages[0]["image_path"]) as sample_image:
        plt.figure(figsize=(6, 8))
        plt.imshow(sample_image)
        plt.axis("off")
        plt.title(f"Sample page 0 from {pages[0]['pdf_file_name']}")
        plt.show()
else:
    print("No pages were generated. Check the PDF path.")


## 4. Load ColPali Model
Load the ColPali processor and model for generating image and text embeddings.


In [ ]:
model_name = "vidore/colpali-v1.2"
processor = ColPaliProcessor.from_pretrained(model_name)

preferred_dtype = torch.bfloat16 if device.type in {"cuda", "mps"} else torch.float32

try:
    model = ColPali.from_pretrained(
        model_name,
        torch_dtype=preferred_dtype,
    )
except Exception as error:
    print(f"Falling back to float32 due to: {error}")
    preferred_dtype = torch.float32
    model = ColPali.from_pretrained(
        model_name,
        torch_dtype=preferred_dtype,
    )

model = model.to(device)
model.eval()

print(f"Model loaded with dtype {preferred_dtype} on {device}.")


## 5. Embed Page Images
Batch process the saved page images through ColPali to create embeddings and metadata.


### Page Embedding Strategy (Mean Pooling)

ColPali produces multiple embedding vectors per page image. Conceptually, it splits the page into many visual regions / patches and gives us an embedding for each region.  
That means for a single PDF page we may get something like 1,000+ vectors, each 128-dimensional.

For our use case we want **page-level retrieval**:
- A search query (text) should bring back the *whole* page image.
- We don't need sub-page / bounding-box precision.

To achieve that, we collapse all region embeddings for a page into **one** page embedding by taking the mean across all region vectors (mean pooling).  
This gives us a single 128-dim vector that represents the entire page's visual/semantic content.

We then:
1. Store exactly one vector per page in Qdrant,
2. Attach metadata (`page_index`, `pdf_file_name`, `image_path`),
3. Later, when we search, Qdrant returns the most relevant page.

This keeps Qdrant small (1 point per page) and aligns with our "retrieve a whole page to display" requirement.


In [ ]:
import torch
from PIL import Image
from pathlib import Path
from tqdm import tqdm


def embed_pages_mean_pooled(
    pages,
    model,
    processor,
    batch_size: int,
    device: torch.device
):
    """
    pages: list[dict] from convert_pdf_to_images(), where each item has:
        {
            "page_index": int,
            "image_path": str,
            "pdf_file_name": str
        }

    Returns: list[dict] where each item is:
        {
            "id": int,                      # stable per page, 1-based
            "embedding": list[float],       # pooled 128-dim vector
            "pdf_file_name": str,
            "page_index": int,
            "image_path": str
        }
    """

    results = []

    # we'll iterate pages in batches
    for start in range(0, len(pages), batch_size):
        batch_slice = pages[start:start + batch_size]

        # load images for this batch
        pil_images = [Image.open(p["image_path"]).convert("RGB") for p in batch_slice]

        # preprocess using the model's processor (same idea as in reference notebook)
        batch_inputs = processor.process_images(pil_images)

        # move tensors to device
        batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()}

        with torch.no_grad():
            # model output shape is typically [B, num_regions, emb_dim]
            model_outputs = model(**batch_inputs)  # tensor or dict-like depending on model
            # if model returns dict-like, adapt accordingly:
            if isinstance(model_outputs, dict):
                page_region_embeddings = model_outputs["embeddings"]
            else:
                page_region_embeddings = model_outputs

            # mean pool across regions for each page
            # page_region_embeddings: [B, R, D]
            # pooled_page_embeddings: [B, D]
            pooled_page_embeddings = page_region_embeddings.mean(dim=1).to("cpu")

        # build result objects
        for i, pooled_vec in enumerate(pooled_page_embeddings):
            src = batch_slice[i]

            results.append({
                "id": src["page_index"] + 1,  # stable 1-based ID
                "embedding": pooled_vec.tolist(),  # turn tensor->python list[float]
                "pdf_file_name": src["pdf_file_name"],
                "page_index": src["page_index"],
                "image_path": str(Path(src["image_path"]))
            })

        # cleanup per-batch to manage memory on Apple Silicon
        del pil_images
        del batch_inputs
        del model_outputs
        del page_region_embeddings
        del pooled_page_embeddings
        torch.mps.empty_cache() if torch.backends.mps.is_available() else None
        try:
            torch.cuda.empty_cache()
        except Exception:
            pass

    return results


# actually run the embedding step
page_embeddings = embed_pages_mean_pooled(
    pages=pages,
    model=model,
    processor=processor,
    batch_size=BATCH_SIZE,
    device=device
)

print(f"Total pages embedded: {len(page_embeddings)}")
print(f"Embedding dim of first page: {len(page_embeddings[0]['embedding'])}")
print("Sample metadata:", {
    "page_index": page_embeddings[0]["page_index"],
    "image_path": page_embeddings[0]["image_path"],
    "pdf_file_name": page_embeddings[0]["pdf_file_name"],
})


## 6. Initialize Qdrant Collection
Create or recreate the Qdrant collection to store page embeddings.


In [ ]:
client = QdrantClient(url=QDRANT_URL)
vector_size = len(page_embeddings[0]["embedding"])

client.recreate_collection(
    collection_name=QDRANT_COLLECTION,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

print(f"Collection '{QDRANT_COLLECTION}' ready with vector size {vector_size}.")


## 7. Upsert Embeddings into Qdrant
Persist the vectors and metadata for each PDF page.


In [ ]:
points = [
    PointStruct(
        id=item["id"],
        vector=item["embedding"],
        payload={
            "pdf_file_name": item["pdf_file_name"],
            "page_index": item["page_index"],
            "image_path": item["image_path"],
        },
    )
    for item in page_embeddings
]

operation_info = client.upsert(collection_name=QDRANT_COLLECTION, points=points)

print(f"Upserted {len(points)} points into collection '{QDRANT_COLLECTION}'.")
if points:
    print("Example payload:", points[0].payload)


## 8. Retrieval Demo
Search the Qdrant collection with a natural-language query and visualize matching pages.


In [ ]:
def search_similar(query_text: str, top_k: int = 5):
    processed_query = processor.process_queries([query_text])
    processed_query = {k: v.to(device) for k, v in processed_query.items()}

    with torch.no_grad():
        outputs = model(**processed_query)

    # Extract query embeddings (shape: [batch_size, num_tokens, embedding_dim])
    if isinstance(outputs, torch.Tensor):
        query_embedding_tensor = outputs
    elif isinstance(outputs, (list, tuple)):
        query_embedding_tensor = outputs[0]
    elif hasattr(outputs, "embeddings"):
        query_embedding_tensor = outputs.embeddings
    elif hasattr(outputs, "last_hidden_state"):
        query_embedding_tensor = outputs.last_hidden_state
    else:
        raise ValueError("Unexpected model output type for query")

    # Apply mean pooling across tokens to match the page embedding strategy
    # query_embedding_tensor shape: [1, num_tokens, embedding_dim]
    # pooled shape: [1, embedding_dim]
    query_embedding_pooled = query_embedding_tensor.mean(dim=1)[0]
    
    # Convert to list for Qdrant
    query_vector = query_embedding_pooled.detach().to(torch.float32).cpu().tolist()

    results = client.search(
        collection_name=QDRANT_COLLECTION,
        query_vector=query_vector,
        limit=top_k,
        with_payload=True,
    )

    matches = []
    for rank, result in enumerate(results, start=1):
        payload = result.payload
        image_path = payload["image_path"]
        with Image.open(image_path) as image:
            plt.figure(figsize=(6, 8))
            plt.imshow(image)
            plt.axis("off")
            plt.title(f"score={result.score:.4f} | page={payload['page_index']} | file={payload['pdf_file_name']}")
            plt.show()

        matches.append(
            {
                "rank": rank,
                "score": result.score,
                "payload": payload,
            }
        )

    return matches

In [ ]:
# Example query; update with a relevant question for your PDF
# matches = search_similar("What is the main topic of this document?", top_k=3)
# matches


## 9. Answer Generation Stub
Outline how to hand retrieved images to a vision-language model for question answering.


### Vision-Language Answering with Qwen2-VL

After we retrieve the most relevant PDF page(s) using ColPali + Qdrant, we can ask a Vision-Language Model (VLM) to read that page image and answer the user's question.

Why we need a VLM:
- Our pipeline stores pages as images (not just plain text).
- Sometimes OCR is missing or unreliable.
- We want a natural-language answer grounded in the page content, including tables, diagrams, stamps, etc.

Flow:
1. User asks a question.
2. We embed the question, search Qdrant, and get back the top matching page image.
3. We send that page image + the user's question into a local Qwen2-VL model.
4. The model generates an answer using ONLY what it "sees" in that page.

This section shows how to load Qwen2-VL locally and run a question-answer step against the best-matching page.


In [ ]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoProcessor

# Configure device for VLM
vlm_device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

VLM_MODEL_NAME = "Qwen/Qwen2-VL-2B-Instruct"  # keep this as a string variable so I can change it later

# Load processor / tokenizer / image processor
vlm_processor = AutoProcessor.from_pretrained(VLM_MODEL_NAME)

# Load model
vlm_model = AutoModelForCausalLM.from_pretrained(
    VLM_MODEL_NAME,
    torch_dtype=torch.float16 if torch.backends.mps.is_available() else torch.float32,
    device_map=None,
)
vlm_model = vlm_model.to(vlm_device)
vlm_model.eval()

print("VLM loaded on", vlm_device)


In [ ]:
def answer_question_with_vlm(query_text: str, retrieved_results):
    """
    Use the local Qwen VLM to answer the user's query by looking directly at
    the most relevant PDF page image we retrieved from Qdrant.
    """

    if not retrieved_results:
        return "No results found to answer from."

    best = retrieved_results[0]
    image_path = best["payload"]["image_path"]

    image = Image.open(image_path).convert("RGB")

    conversation = [
        {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant that answers questions using only the provided document page."}]},
        {"role": "user", "content": [
            {"type": "text", "text": query_text},
            {"type": "image"}
        ]}
    ]

    prompt = vlm_processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = vlm_processor(
        text=prompt,
        images=[image],
        return_tensors="pt"
    )
    inputs = {k: v.to(vlm_device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}

    with torch.no_grad():
        generated_ids = vlm_model.generate(
            **inputs,
            max_new_tokens=256
        )

    generated_ids = generated_ids[:, inputs["input_ids"].shape[-1]:]

    answer = vlm_processor.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )[0]

    return answer


In [ ]:
# Example: ask something about the document
user_query = "Summarize the main purpose of this document."

retrieved = search_similar(user_query, top_k=3)
print("Top match page index:", retrieved[0]["payload"]["page_index"])
print("Image path:", retrieved[0]["payload"]["image_path"])

final_answer = answer_question_with_vlm(user_query, retrieved)
print("\n=== Model Answer ===\n")
print(final_answer)


## 10. Cleanup
Release model resources so the notebook can be rerun without a kernel restart.


In [ ]:
try:
    del model
except NameError:
    pass

try:
    del processor
except NameError:
    pass

try:
    del page_embeddings
except NameError:
    pass

try:
    del vlm_model
except NameError:
    pass

try:
    del vlm_processor
except NameError:
    pass

try:
    del vlm_device
except NameError:
    pass

_ = torch.mps.empty_cache() if torch.backends.mps.is_available() else None
try:
    torch.cuda.empty_cache()
except Exception:
    pass

import gc
gc.collect()
print("VLM + retrieval models cleaned up.")
